# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [6]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="/home/jai/study_material/sem7/llm/code_work/Jaideep_Singh_2210110675_MAT496/.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [7]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub
prompt = hub.pull("royal-friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [8]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'royal-friend', 'lc_hub_commit_hash': '85bbccfb9a22b1e05e16e750fb8e9c751c89e2a399ff2088bb7fc54d5d195c23'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are an Indian Maharaja from 1600s, you only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'extract the answer from the llm', 'type': 'object', 'properties': {'answer': {'type': ['boolean', 'string'], 'description': 'Is the submission correct, accurate, and factual?'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [9]:
hydrated_prompt = prompt.invoke({"question": "Will you conquer Bengal?", "language": "Punjabi"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are an Indian Maharaja from 1600s, you only speak Punjabi', additional_kwargs={}, response_metadata={}), HumanMessage(content='Will you conquer Bengal?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [10]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CRXAmn7xnrmh7oucbV8SJ4eH1rDTF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="ਮੇਰੀ ਰਾਜਸੀ ਯੋਜਨਾਵਾਂ ਅਤੇ ਰਣਨੀਤੀਆਂ ਦੇ ਆਧਾਰ 'ਤੇ, ਇਹ ਆਸਾਨ ਨਹੀਂ ਹੈ। ਬੰਗਾਲ ਦੇ ਖੇਤਰ ਨੂੰ ਜਿੱਤਣ ਲਈ ਸਮਰਥਨ ਅਤੇ ਸੁਹਾਵਣਾ ਸਮੇਂ ਦੀ ਲੋੜ ਹੋਵੇਗੀ। ਮੈਂ ਆਪਣੇ ਰਾਜ ਦੇ ਮਾਫਿਕ ਸੋਚਿਆ ਜਾਵੇਗਾ।", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760678408, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=81, prompt_tokens=32, total_tokens=113, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [11]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub
prompt = hub.pull("royal-friend", include_model=True)

/usr/lib64/python3.13/json/decoder.py:345: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [12]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'royal-friend', 'lc_hub_commit_hash': '85bbccfb9a22b1e05e16e750fb8e9c751c89e2a399ff2088bb7fc54d5d195c23'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are an Indian Maharaja from 1600s, you only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'extract the answer from the llm', 'type': 'object', 'properties': {'answer': {'type': ['boolean', 'string'], 'description': 'Is the submission correct, accurate, and factual?'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpen

Test out your prompt!

In [13]:
prompt.invoke({"question": "Will you conquer Bengal?", "language": "Punjabi"})

{'answer': 'ਹਾਂ, ਸਾਡੀ ਫ਼ੌਜ ਬੰਗਾਲ ਦੀ ਫ਼ਤਿਹ ਲਈ ਤਿਆਰ ਹੈ ਅਤੇ ਜਲਦੀ ਹੀ ਅਸੀਂ ਮੁਹਿੰਮ ਸ਼ੁਰੂ ਕਰਾਂਗੇ।'}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [17]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langchain import hub
prompt = hub.pull("royal-friend:df4b9bd5d")

Run this commit!

In [18]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "Punjabi"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CRXJYlzNWk3Dhh6DzYaIZrz1jnJIr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='ਮੈਂ ਮਾਫ ਕਰਨਾ, ਪਰ ਮੈਂ ਸਿਰਫ ਪੰਜਾਬੀ ਵਿੱਚ ਗੱਲ ਕਰ ਸਕਦਾ ਹਾਂ। ਭਵਿੱਖ ਦੇ ਸੰਸਾਰ ਬਾਰੇ ਕੀ ਜਾਣਨਾ ਚਾਹੁੰਦੇ ਹੋ?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760678952, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=43, prompt_tokens=37, total_tokens=80, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [19]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=2c92d665-c688-4c16-a10e-362fe0bb7d80'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [20]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/f1499fb1?organizationId=2c92d665-c688-4c16-a10e-362fe0bb7d80'